In [1]:
#!pip install gradio
import gradio as gr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
import gradio as gr
import numpy as np


/shared-libs/python3.10/py/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!pip install scikit-learn==1.5.0

In [3]:
#!pip install scikit-learn==1.0.2 catboost==1.2.5 joblib==1.2.0 gradio==4.37.1

In [4]:
import sklearn
import catboost
import joblib
import gradio

print(f"scikit-learn version: {sklearn.__version__}")
print(f"catboost version: {catboost.__version__}")
print(f"joblib version: {joblib.__version__}")
print(f"gradio version: {gradio.__version__}")

scikit-learn version: 1.5.0
catboost version: 1.2.5
joblib version: 1.2.0
gradio version: 4.37.1


In [5]:
import os

print(os.listdir('.'))


['.deepnote', '1_catboost_interval.sav', '1_lgbm_interval.sav', '1_regression_interval.sav', '27_var_hyper_catboost_interval.sav', '2_LGBMRegressor_hyperopt_birthweight.pkl', '2_catboost_hyperopt_birthweight.pkl', '2_catboost_interval.joblib', '2_catboost_interval.sav', '2_lgbm_interval.sav', '2_lgbm_interval_max.sav', '2_regression_interval.sav', '3_catboost_bcu.joblib', '3_catboost_interval_bcu.joblib', '3_lgbm_bcu.joblib', 'EDA_report.html', '__pycache__', 'car_dataframe.csv', 'car_file_to_apply_ohe_feature_selection.csv', 'cat_model.joblib', 'catboost_hyperopt_birthweight.pkl', 'catboost_info', 'catboost_optuna_3.joblib', 'deepnote_exports', 'flagged', 'less_var_hyper_catboost_interval.sav', 'lgbm_hyperopt_birthweight.pkl', 'ml_utils.py', 'model_cloudpickle.pkl', 'requirements-20240620-161239.txt', 'requirements.txt', 'use_to_test.csv', 'use_to_train.csv', 'use_to_val.csv']


In [6]:
#Paso 2: Importar las bibliotecas y cargar el modelo


import joblib
import os

print(os.listdir('.'))

# Cargar el modelo desde el archivo
model_path = './3_catboost_interval_bcu.joblib'
model = joblib.load(model_path)

['.deepnote', '1_catboost_interval.sav', '1_lgbm_interval.sav', '1_regression_interval.sav', '27_var_hyper_catboost_interval.sav', '2_LGBMRegressor_hyperopt_birthweight.pkl', '2_catboost_hyperopt_birthweight.pkl', '2_catboost_interval.joblib', '2_catboost_interval.sav', '2_lgbm_interval.sav', '2_lgbm_interval_max.sav', '2_regression_interval.sav', '3_catboost_bcu.joblib', '3_catboost_interval_bcu.joblib', '3_lgbm_bcu.joblib', 'EDA_report.html', '__pycache__', 'car_dataframe.csv', 'car_file_to_apply_ohe_feature_selection.csv', 'cat_model.joblib', 'catboost_hyperopt_birthweight.pkl', 'catboost_info', 'catboost_optuna_3.joblib', 'deepnote_exports', 'flagged', 'less_var_hyper_catboost_interval.sav', 'lgbm_hyperopt_birthweight.pkl', 'ml_utils.py', 'model_cloudpickle.pkl', 'requirements-20240620-161239.txt', 'requirements.txt', 'use_to_test.csv', 'use_to_train.csv', 'use_to_val.csv']


In [7]:
#Paso 3: Definir la función de predicción
def predecir_cotizacion(horsepower, displacement, torque, wheels, km, age, navigation_system, rear_sensor, push_start, turbo, body_type):
    """
    Función para predecir la cotización del coche.
    
    Parámetros:
    Las características del coche.

    Retorna:
    str: Mensaje con los valores "lower" y "upper" de la cotización del coche en pesos redondeados.
    """
    try:
        navigation_system = 1 if navigation_system == "Sí" else 0
        rear_sensor = 1 if rear_sensor == "Sí" else 0
        push_start = 1 if push_start == "Sí" else 0
        turbo = 1 if turbo == "Sí" else 0

        # Crear el DataFrame con las características
        data = {
            'horsepower': [horsepower],
            'displacement': [displacement],
            'torque': [torque],
            'wheels': [wheels],
            'km': [km],
            'age': [age],
            'navigation_system': [navigation_system],
            'rear_sensor': [rear_sensor],
            'push_start': [push_start],
            'turbo': [turbo],
            'body_type': [body_type]  # Incluir body_type como una columna categórica
        }

        df = pd.DataFrame(data)

        predicciones = model.predict(df)

        # Obtener los valores "lower" y "upper" y convertirlos a pesos
        lower, upper = predicciones[0][0], predicciones[0][1]

        # Redondear los resultados a dos decimales y formatearlos como moneda
        lower = f"${lower:,.2f}"
        upper = f"${upper:,.2f}"

        return f"The car can cost between {lower} and {upper}"

    except Exception as e:
        print(f"Error: {e}")
        return str(e), str(e)

In [10]:
#Paso 4: Definir las entradas y salidas para Gradio


sliders = [
    gr.Slider(label="Horsepower", minimum=66, maximum=557, step=1),
    gr.Slider(label="Displacement", minimum=0, maximum=6.2, step=0.1),
    gr.Slider(label="Torque", minimum=67, maximum=561, step=1),
    gr.Slider(label="Wheels", minimum=14, maximum=22, step=1),
    gr.Slider(label="Distance (km)", minimum=4300, maximum=173850, step=1000),
    gr.Slider(label="Age (years)", minimum=3, maximum=15, step=1)
]

dropdowns = [
    gr.Dropdown(label="Navigation System", choices=["Sí", "No"]),
    gr.Dropdown(label="Rear Sensor", choices=["Sí", "No"]),
    gr.Dropdown(label="Push Start", choices=["Sí", "No"]),
    gr.Dropdown(label="Turbo", choices=["Sí", "No"]),
    gr.Dropdown(label="Body Type", choices=["Sedan", "Suv", "Hatchback", "Minivan", "Pickup", "Van", "Coupe", "Targa", "Convertible", "Wagon"])
]


inputs = sliders + dropdowns


outputs = gr.Textbox(label="Estimated price")



In [9]:
#Paso 5: Crear y lanzar la interfaz de Gradio
# Crea la interfaz de Gradio

iface = gr.Interface(
    fn=predecir_cotizacion,
    inputs=inputs,
    outputs=outputs,
    title="Car pricing",
    description="Enter the car's features to get a quote.",
    allow_flagging="never",
    live=False  
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://84dea5f6881b21fa96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a817877f-bc24-4404-a1bc-b1f4f599a592' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>